In [3]:
import yaml
import pandas as pd

folder_location = 'C:/Users/ttrol/CodingProjects/MatNet/scripts/webscraper/results_files/'
filename = 'AISI_steels.yaml'
file_location = folder_location + filename

with open(file_location, "r") as stream:
    try:
        raw_materials = yaml.safe_load(stream)
        print('Raw Size: ', len(raw_materials))
        raw_df = pd.DataFrame(raw_materials)
    except yaml.YAMLError as exc:
        print(exc)

Raw Size:  1813


In [ ]:
import requests
from bs4 import BeautifulSoup

session = requests.Session()
    # session.cookies.set_policy(BlockAll())
    headers =  {
       'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.36',
    }
    
    entry = ('http://customer-%s:%s@%s' %
        (USERNAME, PASSWORD, ENDPOINT))
    query = {
        'http': entry,
        'https': entry,
    }

    proxies = query

    for i in range(5):
        response = session.get(
            url='https://ip.oxylabs.io',
            proxies= proxies,
            verify=True,
            headers=headers
        )
        soup = BeautifulSoup(response.content,'lxml')
        print(soup.get_text().strip())

        response = session.get(
            headers=headers,
            url='https://matweb.com/search/DataSheet.aspx?MatGUID=66575ff2cd5249c49d76df15b47dbca4',
            proxies= proxies,
            verify=True
        )

        soup = BeautifulSoup(response.content,'lxml')
        print(soup.title)